The model is based on two factors. The first is value over placement(VOR), which is calculated by stats scored by players in 2020 and determines how many projected fantasy points we might lose if we do not draft the player in that specific round. The other factor is average draft position(ADP), which we acquire from “fantasyfootballcalculator.com”. By comparing the ADP and VOR rankings, we are able to project which players might give us a higher value than their ADP suggests, giving us a more holistic view on player drafting.

Normally when drafting players we instinctively select the player with the highest projected fantasy points available to us, but this method is often short sighted. For example if there are 4 players remaining. QB1 has 400 points, QB2 has 375 points; RB1 has 350 points and RB2 has 300 points. If we draft QB1 now and lose out on RB1 our next option will be RB2, this could cost us 50 projected points. But by drafting RB1 now we will only lose 25 projected points since QB2 will probably be available next round. And that is the concept of VOR. By looking at my draft table with VOR rankings, we will be able to easily determine which players to draft without potentially losing more points.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/fantasydatapros/data/master/fantasypros/fp_projections.csv')

df.head()

HTTPError: ignored

In [ ]:
type(df)

In [ ]:
df.head(10)

In [ ]:
df = df.iloc[:, 1:]
df.head()

In [ ]:
df.columns

In [ ]:
', '.join(df.columns)

In [ ]:
scoring_weights = {
    'receptions': 1, #PPR
    'receiving_yds': 0.1,
    'receiving_td': 6,
    'FL': -2, #fumble lost
    'rushing_yds': 0.1,
    'rushing_td': 6,
    'passing_yds': 0.04,
    'passing_td': 4,
    'int': -2
}

df['FantacyPoint'] = (
    df['Receptions']*scoring_weights['receptions'] + df['ReceivingYds']*scoring_weights['receiving_yds'] + \
    df['ReceivingTD']*scoring_weights['receiving_td'] + df['FL']*scoring_weights['FL'] + \
    df['RushingYds']*scoring_weights['rushing_yds'] + df['RushingTD']*scoring_weights['rushing_td'] + \
    df['PassingYds']*scoring_weights['passing_yds'] + df['PassingTD']*scoring_weights['passing_td'] + \
    df['Int']*scoring_weights['int'] )

df.head()

In [ ]:
rb_df = df.loc[df['Pos'] == 'RB']

rb_df.head()

In [ ]:
base_columns = ['Player', 'Team', 'Pos']
rushing_columns = ['FantasyPoints', 'Receptions', 'ReceivingYds', 'ReceivingTD', 'RushingAtt', 'RushingYds', 'RushingTD', 'FL']
rb_df = df.loc[df['Pos'] == 'RB', base_columns + rushing_columns]

rb_df.head()

In [ ]:
rb_df.sort_values(by = 'RushingYds', ascending = False).head(15)

In [ ]:
rb_df.describe().transpose()

In [ ]:
rb_df['RushingAtt'][:10]

In [ ]:
rb_df['RushingAtt'].max()

In [ ]:
rb_df['RushingAtt'].std() #standard deviation of a series column

In [ ]:
rb_df['RushingAtt'].min()

In [ ]:
rb_df['RushingAtt'].quantile(0.75)

In [ ]:
rb_df['RushingAtt'].quantile(0.25)

In [ ]:
rb_df['RushingTDRank'] = rb_df['RushingAtt'].rank(ascending = False)

rb_df.sort_values(by = 'RushingTDRank').head(5)

In [ ]:
rb_df['RushingAtt'].value_counts()

In [ ]:
import seaborn as sns
sns.set_style('whitegrid')
sns.distplot(rb_df['RushingAtt'])

In [ ]:
rb_df.values

In [ ]:
adp_df = pd.read_csv('https://raw.githubusercontent.com/fantasydatapros/data/master/fantasypros/adp/PPR_ADP.csv')

adp_df.head()

In [ ]:
adp_df = adp_df.iloc[:, 1:]

adp_df.head()

In [ ]:
adp_df['ADP RANK'] = adp_df['AVG'].rank()

adp_df.head()

In [ ]:
adp_df_cutoff = adp_df[:100]

adp_df_cutoff.shape

In [ ]:
replacement_players = {
    'RB':'',
    'QB':'',
    'WR':'',
    'TE':''
}

for _, row in adp_df_cutoff.iterrows():

  position = row['POS']
  player = row['PLAYER']

  if position in replacement_players:
    replacement_players[position] = player

replacement_players

In [ ]:
df = df[['Player', 'Pos', 'Team', 'FantasyPoints']]
df.head()

In [ ]:
replacement_values = {}

for position, player_name in replacement_players.items():

 player = df.loc[df['Player'] == player_name]

 replacement_values[position] = player['FantasyPoints'].tolist()[0]

replacement_values

In [ ]:
pd.set_option('chained_assignment', None)

df = df.loc[df['Pos'].isin(['QB', 'RB', 'WR', 'TE'])]

df['VOR'] = df.apply(
    lambda row: row['FantasyPoints'] - replacement_values.get(row['Pos']), axis = 1
)
df.head()

In [ ]:
pd.set_option('display.max_rows', None)

df['VOR Rank'] = df['VOR'].rank(ascending = False)
df.sort_values(by='VOR', ascending = False).head(100)

In [ ]:
df.groupby('Pos')['VOR'].describe()

In [ ]:
df['VOR'] = df['VOR'].apply(lambda x: (x - df['VOR'].min()) / (df['VOR'].max() - df['VOR'].min()))

df = df.sort_values(by='VOR Rank')
df.head(100)

In [ ]:
import seaborn as sn

num_teams = 12
num_spots = 16 # 1 QB, 2RB, 2WR, 1TE, 1FLEX, 1K, 1DST, 7 BENCH
draft_pool = num_teams * num_spots

df_copy = df[:draft_pool]

sns.boxplot(x=df_copy['Pos'], y=df_copy['VOR']);

In [ ]:
df = df.rename({
    'VOR': 'Value',
    'VOR Rank': 'Value Rank'
}, axis = 1)

adp_df = adp_df.rename({
    'PLAYER': 'Player',
    'POS': 'Pos',
    'AVG': 'Average ADP',
    'ADP RANK': 'ADP Rank'
}, axis = 1)

final_df = df.merge(adp_df, how = 'left', on = ['Player', 'Pos'])

final_df.head()

In [ ]:
final_df['Diff in ADP and Value'] = final_df['ADP Rank'] - final_df['Value Rank']
final_df.head()

In [ ]:
draft_pool = final_df.sort_values(by = 'ADP Rank')[:196]

rb_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'RB']
qb_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'QB']
wr_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'WR']
te_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'TE']

# top 10 RB sleepers for this year's draft
rb_draft_pool.sort_values(by = 'Diff in ADP and Value', ascending = False)[:10]

In [ ]:
# top 10 RB overvalued for this year's draft
rb_draft_pool.sort_values(by='Diff in ADP and Value', ascending = True)[:10]

In [ ]:
# top 10 WR sleepers for this year's draft
wr_draft_pool.sort_values(by='Diff in ADP and Value', ascending = False)[:10]

In [ ]:
# top 10 WR overvalued for this year's draft
wr_draft_pool.sort_values(by='Diff in ADP and Value', ascending = True)[:10]

In [ ]:
# top 10 TE sleepers for this year's draft
te_draft_pool.sort_values(by='Diff in ADP and Value', ascending = False)[:10]

In [ ]:
# top 10 TE overvalued for this year's draft
te_draft_pool.sort_values(by='Diff in ADP and Value', ascending = True)[:10]

In [ ]:
# top 10 QB sleepers for this year's draft
qb_draft_pool.sort_values(by='Diff in ADP and Value', ascending = False)[:10]

In [ ]:
# top 10 QB overvalued for this year's draft
qb_draft_pool.sort_values(by='Diff in ADP and Value', ascending = True)[:10]